In [1]:
import numpy as np
from data_loading import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
img = load_nii('../data/liver_only/image/10000005_CT_0mean1std_notCropped_2mmVoxel.nii.gz')
mask =load_nii('../data/multilabel/mask/10000005_labelmask_notCropped_2mmVoxel.nii.gz') 
image_shape = np.shape(img)
patch_shape = (64, 64, 64)
extraction_step = 8
patches = extract_patches(img, patch_shape=patch_shape,extraction_step=extraction_step)
mask_patches = extract_patches(mask, patch_shape=patch_shape,extraction_step=extraction_step)
print(image_shape)

(210, 210, 660)


In [3]:
path = write_nii(img, 'test.nii')

In [ ]:


trained_model = unet3d_3blocks(pretrained_weights='./saved_models/liver_segmentation.h5',input_size = (*patch_size, 1), num_classes=num_classes)
val_preds = trained_model.predict(val_gen)
print(np.shape(val_preds))

In [ ]:
n=np.size(patches, axis=0)
z = 45
plt.figure(figsize=[9,9])
for i in range(n):
    for j in range(n):
        plt.subplot(n,n,n*i+j+1)
        plt.imshow(patches[i,j,z,:,:,0],vmin=-2, vmax=2,cmap='bone')

In [4]:
def build_from_patches(patches, image_shape, extraction_step):
    '''reconstruct a single 3D image volume from a bunch of patches
    patches has shape [i,j,k,x,y,z] where i,j,k are number of patches in each dimension and x,y,z is patch shape
    image_size is tuple (3,) - the size of the whole image
    extraction_step is scalar - the extraction step used to generate the patches'''
    
    img = np.zeros(image_shape)
    patch_size = np.shape(patches)[-3:]

    for i in range(np.size(patches, 0)):
        for j in range(np.size(patches, 1)):
            for k in range(np.size(patches,2)):
                img[i*extraction_step:i*extraction_step+patch_size[0],
                   j*extraction_step:j*extraction_step+patch_size[1],
                   k*extraction_step:k*extraction_step+patch_size[2]] = patches[i,j,k,:,:,:]
    return img

In [5]:
img = build_from_patches(mask_patches, image_shape, extraction_step)

In [6]:
write_nii(img, 'test.nii')

'test.nii'

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(img[:,:,370], vmin=0, vmax=5,cmap='bone')